# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [69]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [70]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [71]:
def build_classifiers():
    linreg = LinearRegression()
    knn = KNeighborsClassifier()
    svm = SVC()
    dt = DecisionTreeClassifier()
    nb = GaussianNB()
    qda = QuadraticDiscriminantAnalysis()

    linreg.fit(data_set, labels)
    knn.fit(data_set, labels)
    svm.fit(data_set, labels)
    dt.fit(data_set, labels)
    nb.fit(data_set, labels)
    qda.fit(data_set, labels)

    return [linreg, knn, svm, dt, nb, qda]

In [74]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    # print(output)
    output = np.array(output).reshape((130,3))
    
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier() # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [75]:
from itertools import permutations
from operator import itemgetter

classifiers = build_classifiers()

results = sorted(
    [(
        " ".join(type(c).__name__ for c in perm),
        accuracy_score(test_labels, build_stacked_classifier(perm))
    ) for perm in permutations(classifiers, r=3)],
    key=itemgetter(1),
    reverse=True
)
for r in results:
    print(r)

('DecisionTreeClassifier KNeighborsClassifier QuadraticDiscriminantAnalysis', 0.9)
('GaussianNB KNeighborsClassifier QuadraticDiscriminantAnalysis', 0.9)
('KNeighborsClassifier SVC GaussianNB', 0.85)
('KNeighborsClassifier SVC QuadraticDiscriminantAnalysis', 0.85)
('KNeighborsClassifier GaussianNB QuadraticDiscriminantAnalysis', 0.85)
('KNeighborsClassifier QuadraticDiscriminantAnalysis DecisionTreeClassifier', 0.85)
('SVC KNeighborsClassifier GaussianNB', 0.85)
('SVC KNeighborsClassifier QuadraticDiscriminantAnalysis', 0.85)
('GaussianNB KNeighborsClassifier DecisionTreeClassifier', 0.85)
('GaussianNB SVC KNeighborsClassifier', 0.85)
('QuadraticDiscriminantAnalysis KNeighborsClassifier DecisionTreeClassifier', 0.85)
('KNeighborsClassifier SVC DecisionTreeClassifier', 0.8)
('KNeighborsClassifier DecisionTreeClassifier QuadraticDiscriminantAnalysis', 0.8)
('KNeighborsClassifier GaussianNB DecisionTreeClassifier', 0.8)
('SVC KNeighborsClassifier DecisionTreeClassifier', 0.8)
('SVC Gaussi

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [56]:
# import numpy as np
# from sklearn.tree import DecisionTreeClassifier

# # prepare data set

# def generate_data(sample_number, feature_number, label_number):
#     data_set = np.random.random_sample((sample_number, feature_number))
#     labels = np.random.choice(label_number, sample_number)
#     return data_set, labels

# labels = 2
# dimension = 2
# test_set_size = 1000
# train_set_size = 5000
# train_set, train_labels = generate_data(train_set_size, dimension, labels)
# test_set, test_labels = generate_data(test_set_size, dimension, labels)

# # init weights
# number_of_iterations = 10
# weights = np.ones((test_set_size,)) / test_set_size


# def train_model(classifier, weights):
#     return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

# def calculate_error(model):
#     predicted = model.predict(test_set)
#     I=calculate_accuracy_vector(predicted, test_labels)
#     Z=np.sum(I)
#     return (1+Z)/1.0

import numpy as np
from sklearn.tree import DecisionTreeClassifier


# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels


labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)


def calculate_error(model):
    predicted = model.predict(test_set)
    I = calculate_accuracy_vector(predicted, test_labels)
    Z = np.sum(I)
    return (1 + Z) / 1.0

Fill the two functions below:

In [57]:
def calculate_accuracy_vector(predicted, labels):
    return (predicted == labels).astype(int)

def set_new_weights(model):
    I = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (1 + I) / (1 + I).sum()

Train the classifier with the code below:

In [58]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00132275 0.00132275 0.00066138 0.00066138 0.00132275 0.00132275
 0.00132275 0.00132275 0.00132275 0.00132275 0.00066138 0.00066138
 0.00132275 0.00132275 0.00132275 0.00066138 0.00066138 0.00132275
 0.00066138 0.00132275 0.00066138 0.00132275 0.00066138 0.00066138
 0.00066138 0.00132275 0.00066138 0.00132275 0.00066138 0.00132275
 0.00066138 0.00066138 0.00132275 0.00132275 0.00132275 0.00066138
 0.00066138 0.00066138 0.00132275 0.00066138 0.00066138 0.00132275
 0.00066138 0.00132275 0.00132275 0.00132275 0.00066138 0.00132275
 0.00066138 0.00066138 0.00132275 0.00132275 0.00066138 0.00132275
 0.00132275 0.00132275 0.00132275 0.00132275 0.00066138 0.00066138
 0.00066138 0.00132275 0.00132275 0.00066138 0.00066138 0.00132275
 0.00132275 0.00066138 0.00066138 0.00132275 0.00132275 0.00132275
 0.00132275 0.00066138 0.00066138 0.00066138 0.00132275 0.00066138
 0.00132275 0.00132275 0.00132275 0.00132275 0.00066138 0.00132275
 0.00132275 0.00066138 0.00132275 0.00066138 0.00066138 0.0013

Set the validation data set:

In [59]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [60]:
def get_prediction(x):
    output = np.array([classifier.predict(x) for classifier in classifiers])
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        predicted.append(np.argmax(np.bincount(classified)))
    return predicted

Test it:

In [61]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
